In [1]:
from scipy import optimize, signal, io
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def f(x, a, b, c, d):
    # y = a*sin(b*x + c) + d
    return a*np.sin(b*x + c) + d

def guess_amplitude(y):
    return (np.max(y) - np.min(y))/2

def guess_period(x, y, width=None):
    peaks, properties = signal.find_peaks(y, width=width)
    print(f"{peaks=}")
    if np.size(peaks) >= 2:
        mean_peak_dist = int(np.mean(np.diff(peaks)))
        diff_x = x[mean_peak_dist] - x[0]
        f = 2*np.pi/diff_x
    else:
        diff_x = x[-1] - x[0]
        f = 2*np.pi/diff_x
        
    return f
    
def find_initial_params(x, y, width_peaks=None):
    # Some decent initial guesses
    a = guess_amplitude(y)
    b = guess_period(x,y, width=width_peaks)
    return a, b, 1, 1

def fit(f, x, y, width_peaks=None):
    sol, cov = optimize.curve_fit(
        f, x, y, p0=find_initial_params(x, y, width_peaks=width_peaks) 
    ) # bounds=(-inf, inf)
    print(sol)
    err = np.sqrt(np.diag(cov))
    return sol, cov, err

In [3]:
def test_cases(i):
    if i == 1:
        return 10, 0.3, 0.5, 5
    if i == 2:
        return 10, 10, 10, 10
    if i == 3:
        return 10, 0.1, 10, 10
    if i == 4:
        return 10, 1, 10, 10

def real_cases():
    with open('x.dat') as f:
        x_lines = f.readlines()
    with open('y.dat') as f:
        y_lines = f.readlines()
    with open('z.dat') as f:
        z_lines = f.readlines()
        
    #x = np.loadtxt('x.dat', delimiter=';')
    #y = np.loadtxt('y.dat', delimiter=';')
    #z = np.loadtxt('z.dat', delimiter=';')
    
    return x_lines, y_lines, z_lines


In [4]:
## Make some test data
#t_sample = np.linspace(0,10, 100)
#a, b, c, d = test_cases(3)
#y_sample = f(t_sample, a, b, c, d )
# Add noise
#y_sample += np.random.normal(0, 0.2, len(y_sample))
# Make some test data
x_lines, y_lines, z_lines = real_cases()


In [23]:
case = -12
x = np.fromstring(x_lines[case], sep=';')
y = np.fromstring(y_lines[case], sep=';')
z = np.fromstring(z_lines[case], sep=';')
#if len(x.shape) > 1:   
#    t_sample = np.array([i for i in range(len(x[0]))])
#    y_sample = y[0]
#else:
t_sample = np.array([i for i in range(len(x))])
y_sample = x
y_hat = signal.savgol_filter(y_sample, len(y_sample), 7)
sol, cov, err = fit(f, t_sample, y_hat)
y_fit = f(t_sample, sol[0], sol[1], sol[2], sol[3])

plt.plot(t_sample, y_sample, 'bo' ,label='data')
#plt.plot(t_sample, y_hat, label='filtered')
plt.plot(t_sample, y_fit, 'r', label='fit')
plt.legend(loc="lower left")
plt.show()

IndexError: list index out of range

In [ ]:
#plt.plot(t_sample, y_sample)
#yhat = signal.savgol_filter(y_sample, len(y_sample), 7)
#plt.plot(t_sample, yhat)
#plt.show()

In [ ]:
#sol, cov, err = fit(f, t_sample, yhat, width_peaks=None)

In [ ]:
plt.plot(t_sample, y_sample)
plt.plot(t_sample, y_fit)
plt.show()